Загрузка даннных

In [26]:
import pandas as pd
import numpy as np
import random

import os
from src.panda import ReestrData

dataset = ReestrData()
dataset.config()
dataset = dataset.create_df()

ModuleNotFoundError: No module named 'src'

In [2]:
#Очистить данные о предыдущих лицензиях
dataset['prew_'] = dataset['prew_full'].str.replace('\n', ':', regex=True).str.replace(' от \d\d\.\d\d', '', regex=True)
#Проверка целостности 
assert dataset['prew_'].count() == dataset['prew_full'].count(), 'Incorrect prew extract'

#Извлечь все предыдущие лицензии и их год
pattern = "(?P<old_lic>[А-Я]{3}\d*[А-Я]{2})(?:.)(?P<old_year>\d{4})"
df_left = dataset['prew_'].str.extractall(pattern).droplevel(1).astype(dtype={'old_year':'int'})

#Добавить все текущие лицензии и их год
df_right = dataset.loc[dataset['Last'] == True, ['Year']].astype('int')

#Таблица для связи текущих лицензий с предыдущими
lookup_table = df_left.merge(df_right, how='right', left_index=True, right_index=True)
lookup_table = lookup_table[['Year', 'old_lic', 'old_year']]    #.to_excel(excel_writer='data/test.xlsx')

#Проверка соответствия индекса лукап таблицы с исходником
assert len(lookup_table.index.unique()) == len(dataset[dataset['Last'] == True].index), "Error"

In [3]:
lookup_table

,Year,old_lic,old_year
num,,,
АБН00011НР,1993,NaN,NaN
АБН00220НП,1997,NaN,NaN
АБН00504НП,2009,АБН00472НП,2007.0
АБН00504НП,2009,АБН00385НП,2005.0
АБН13633НР,2006,NaN,NaN
...,...,...,...
ЯКУ16540НР,2019,ЯКУ05186НР,2017.0
ЯКУ16540НР,2019,ЯКУ04590НР,2016.0
ЯКУ16667НП,2020,NaN,NaN


In [4]:
#Создание датафрейма для матрицы соотношения номеров старых лицензий с текущими лицензиями
index = list(set(lookup_table.index.to_list()))
columns = list(set(lookup_table.Year.tolist()))

matrix = pd.DataFrame(data = None, index = index, columns=columns)
matrix

,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
УФА00402НП,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ЯКУ05848НЭ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ХМН13993НП,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
СЛХ02545НП,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
СЛХ010380НЭ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
СМР01580НЭ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ХМН002671НЭ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ОРБ03185НЭ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ИЖВ00226НП,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Алгоритм заполнения датафрейма
for i in lookup_table.itertuples():
    ind = i[0]
    col = i[1]
    old_lic = i[2]
    old_year = i[3]
    matrix.loc[ind, col] = ind
    matrix.loc[ind, old_year] = old_lic

#Заполнение пустых ячеек вправо
matrix = matrix.ffill(axis=1)

In [17]:
if os.path.exists('data/matrix.xlsx'):
    with pd.ExcelWriter(path='data/matrix.xlsx', if_sheet_exists='replace', mode='a') as writer:
        matrix.to_excel(writer, sheet_name='matrix')
        lookup_table.to_excel(writer, sheet_name='lookup_table')
else: 
    matrix.to_excel('data/matrix.xlsx', sheet_name='matrix')
    lookup_table.to_excel('data/matrix.xlsx', sheet_name='lookup_table')

In [16]:
import os

In [11]:
#Тест проверки правильности выгрузки лицензий
matrix.loc[matrix.iloc[random.randint(0,len(matrix.index))].name]

1992.0            NaN
1993.0            NaN
1994.0     ТОМ00021НЭ
1995.0     ТОМ00021НЭ
1996.0     ТОМ00021НЭ
1997.0     ТОМ00066НЭ
1998.0     ТОМ00066НЭ
1999.0     ТОМ00066НЭ
2000.0     ТОМ00066НЭ
2001.0     ТОМ00066НЭ
2002.0     ТОМ00066НЭ
2003.0     ТОМ00066НЭ
2004.0     ТОМ12381НЭ
2005.0     ТОМ12381НЭ
2006.0     ТОМ12381НЭ
2007.0     ТОМ12381НЭ
2008.0     ТОМ12381НЭ
2009.0     ТОМ12381НЭ
2010.0     ТОМ12381НЭ
2011.0     ТОМ12381НЭ
2012.0     ТОМ12381НЭ
2013.0     ТОМ12381НЭ
2014.0     ТОМ01859НЭ
2015.0     ТОМ01859НЭ
2016.0     ТОМ01859НЭ
2017.0     ТОМ01859НЭ
2018.0     ТОМ01859НЭ
2019.0     ТОМ01859НЭ
2020.0     ТОМ01859НЭ
2021.0     ТОМ01859НЭ
2022.0    ТОМ007651НЭ
2023.0    ТОМ007651НЭ
NaN       ТОМ007651НЭ
Name: ТОМ007651НЭ, dtype: object

In [53]:
#Выгрузка дубликатов
lookup_table[lookup_table['old_lic'].duplicated(keep=False) & lookup_table['old_lic'].notna()].sort_values(['old_lic']).to_excel('data/dublicates.xlsx')

____________

In [36]:
test['prew_'].str.extractall(pattern).droplevel(1).astype(dtype={'old_year':'int'}).to_excel('data/test.xlsx')

In [36]:
x = test['prew_'].str.extractall(pattern).droplevel(1)
y = test.loc[test['prew_'].isna(), ['prew_','Year']].rename(columns={'prew_':'old_lic', 'Year':'old_year'})
#pd.concat([x,y], axis=0).astype(dtype={'old_year':'int'}).to_excel('data/test.xlsx')
pd.concat([x,y], axis=0).astype(dtype={'old_year':'int'})

,old_lic,old_year
num,,
АБН00472НП,АБН00385НП,2005
АБН00504НП,АБН00472НП,2007
АБН00504НП,АБН00385НП,2005
АНД00915НЭ,АНД00552НЭ,1996
АНД00916НЭ,АНД00551НЭ,1996
...,...,...
ЯКУ16409НЭ,None,2017
ЯКУ16414НП,None,2017
ЯКУ16667НП,None,2020


Тесты на корректность извлекаемых данных

In [2]:
assert test['prew_full'].count() == test['prev_lic'].count(), 'Ошибка в выгрузке предыдущих лицензий'
assert test['forw_full'].count() == test['forw_lic'].count(), 'Ошибка в выгрузке будущих лицензий'
assert (test['N'].count() == test['coords'].count()) & (test['E'].count() == test['coords'].count()), 'Coords_extract_error'
assert test['owner'].count() == test['owner_full'].count(), 'Name_extract_error'

In [3]:
print(test['prev_lic'].count(), test['prew_full'].count())
print(test['forw_lic'].count(), test['forw_full'].count())
print(test['N'].count(), test['E'].count(), test['coords'].count())
print(test['owner'].count(), test['owner_full'].count())

7934 7934
7928 7928
13395 13395 13395
16105 16105


In [22]:
test.reset_index().loc[random.randint(1, len(test.index)), ['owner', 'owner_full']]

owner         ООО ФОБОССНЕФТЕДОБЫЧА
owner_full    ООО Фобосснефтедобыча
Name: 13359, dtype: object